# Validating and Importing Target Time Series Data

## Obtaining Your Data

A critical requirement to use Amazon Forecast is to have access to time-series data for your selected use case. To learn more about time series data:

1. [Wikipedia](https://en.wikipedia.org/wiki/Time_series)
1. [Toward's Data Science Primer](https://towardsdatascience.com/the-complete-guide-to-time-series-analysis-and-forecasting-70d476bfe775)
1. [O'Reilly Book](https://www.amazon.com/gp/product/1492041653/ref=ppx_yo_dt_b_search_asin_title?ie=UTF8&psc=1)

As an example for this POC guide we are going to select a dataset from the UCI repository of machine learning datasets. This is a great tool for finding datasets for various problems. In this particular case it is traffic data for a given section of interstate highway. More information on the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume)

Your specific data my come from a DB export, an existing spreadsheet, it really does not matter the source. Going forward the files should be uploaded into this notebook and stored in CSV format.

To begin the cell below will complete the following:

1. Create a directory for the data files.
1. Download the sample data into the directory.
1. Extract the archive file into the directory.

In [ ]:
data_dir = "data"
%store data_dir

!mkdir $data_dir
!cd $data_dir && wget https://chriskingpartnershare.s3.amazonaws.com/Metro_Interstate_Traffic_Volume.csv

With the data downloaded, now we will import the Pandas library as well as a few other data science tools in order to inspect the information.

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import json
from time import sleep

# External Dependencies:
import boto3
import pandas as pd

# Local Dependencies:
import util

Next open the file with Pandas and take a look at the contents

In [ ]:
original_data = pd.read_csv(f"{data_dir}/Metro_Interstate_Traffic_Volume.csv")
original_data.head(5)

At this point we can see a few things about the data:

* Holidays seem to be specified
* There is a value for temp, rainfall, snowfall, and a few other weather metrics.
* The time series is hourly
* Our value to predict is `traffic_volume` down at the end.

Amazon Forecast relies on a concept called the target-time-series in order to start making predictions, this has a timestamp, an item identifier, and a value. The timestamp is pretty self explanatory, and the value to predict will be traffic_volume, given this is a singular time series an arbitrary item_id of `all` will be applied later to all entries in the time series file.

The other attributes provided can serve as a basis for related time series components when we get to that much later.

Amazon Forecast also works well to fill in gaps for the target-time-series but not the related data, so before we input our data and get a prediction we should look to see where gaps are, and how we want to structure both inputs to address this issue. 

To get started we will manipulate our starting dataframe to determine the quality and consistency of this dataset.

In [ ]:
target_df = original_data.copy()
target_df.plot()
print("Start Date: ", min(target_df["date_time"]))
print("End Date: ", max(target_df["date_time"]))

Interestingly at this point we do not see any obvious gaps in this plot, but we should still check a bit deeper to confirm this. The next cell gives some basic information on the dataset size.

In [ ]:
target_df.info()

In the cell above we now see a range of October 2012 to nearly October 2018, almost 6 years of hourly data. Given there are around 8700 hours in a year we expect to see 52,000 time series. Immediately here we see 48,204. It looks like some data points are missing, next let us define the index, drop the duplicates and see where we are then.

In [ ]:
target_df.set_index("date_time", inplace=True)
target_df = target_df.drop_duplicates(keep="first")
target_df.info()

That change dropped us to 48,175 unique entries. Given this is traffic data we could be dealing with a missing sensor, construction causing outages, or even severe weather delay damaging the recording equipment. Before we decide on how to fill any gaps, let us first take a look to see where they are, and how large the gaps themselves may be.

We will do this by creating a new dataframe for the entire length of the dataset, that has no missing entries, then joining our data to it, and padding out 0's where anything is missing.

*Note* the periods value below is the total number of entries to make, I cheated and used WolframAlpha to sort out the number of days: https://www.wolframalpha.com/input/?i=days+from+2012-10-02+to+2018-09-30

In [ ]:
total_days = 2190
# Build the index first
idx = pd.date_range(start="10/02/2012", end="09/30/2018", freq="H")

In [ ]:
full_df = pd.DataFrame(index=idx)
full_df.head(3)

In [ ]:
print(full_df.index.min())
print(full_df.index.max())

In [ ]:
# Now perform the join
full_historical_df = full_df.join(target_df, how="outer")
print(full_historical_df.index.min())
print(full_historical_df.index.max())

In [ ]:
# Take a look at 10 random entries
full_historical_df.sample(10)

The sample may or may not have shown values with NaNs or other nulls, in this instance it did but we will still want to look for these NaN entities to confirm if they exist and where they are.

At this point we have done enough work to see where we may have any large portions of missing data. To that end we can plot the data below and see any gaps that may crop up.

In [ ]:
full_historical_df.plot()

This shows a large gap of missing data from late 2014 until mid 2016. If we just wanted to feed in the previously known value this may give us too long of a timeframe of data that is simply not representative of the problem. 

Before making any decisions we will now step through each year and see what the gaps look like starting in 2013 as it is the first full year.

In [ ]:
df_2013 = full_historical_df.loc["2013-01-01":"2013-12-31"]
print(df_2013.index.min())
print(df_2013.index.max())
df_2013.plot()

In [ ]:
df_2014 = full_historical_df.loc["2014-01-01":"2014-12-31"]
print(df_2014.index.min())
print(df_2014.index.max())
df_2014.plot()

In [ ]:
df_2015 = full_historical_df.loc["2015-01-01":"2015-12-31"]
print(df_2015.index.min())
print(df_2015.index.max())
df_2015.plot()

In [ ]:
df_2016 = full_historical_df.loc["2016-01-01":"2016-12-31"]
print(df_2016.index.min())
print(df_2016.index.max())
df_2016.plot()

In [ ]:
df_2017 = full_historical_df.loc["2017-01-01":"2017-12-31"]
print(df_2017.index.min())
print(df_2017.index.max())
df_2017.plot()

In [ ]:
df_2018 = full_historical_df.loc["2018-01-01":"2018-12-31"]
print(df_2018.index.min())
print(df_2018.index.max())
df_2018.plot()

A few things to note here, clearly we are missing a large volume of data in 2014 and 2015 but also there are some missing patches in 2013 as well. 2016 had spotty data initially but 2017 and 2018 look pretty good.

Given that the data is hourly we still have plenty of it within a single year, and an additional 10 months to use for broader validation if we choose to do that.

To note, more advanced approaches like neural network models and Prophet often work very well with > 1k measurements on a given time series. Assuming hourly data (24 measurements per day), that yields around 42 days before we have a solid base of data. Learning over an entire year should be plenty.

Also we need to think about a Forecast horizon or how far into the future we are going to predict at once. Forecast currently limits us to 500 intervals of whatever granularity we have selected. For this exercise we will keep the data hourly and predict 480 hours into the future, or exactly 20 days.

## Building Data Files

Knowing that our above dataframe `full_historical_df` covers the entire time period we care about we start there reducing it to 2017 to end. Then we will use fill forward to plug in any missing holes before splitting into the 3 files described before. 

More info on techniques to patch missing information can be found here: https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.fillna.html 

The risk of filling in values like this is that in smoothing out the data it may cause our predictions to resemble a smoother curve than our historical data. This is why we selected 2017 to 2018 based on the lack of large gaps in the data.

In [ ]:
# Create a copy
target_df = full_historical_df.copy()
# Slice to only 2017 onward
target_df = target_df.loc["2017-01-01":]
# Validate the dates
print(target_df.index.min())
print(target_df.index.max())

In [ ]:
# Fill in any missing data with the method ffill
target_df.ffill()

At this point we have all the data needed to make our target time series file and dataset. While we are doing this we will also make a validation file for later use as well.

### Building The Target Time Series File

In [ ]:
target_time_series_df = target_df.copy()
target_time_series_df = target_time_series_df.loc["2017-01-01":"2017-12-31"]
# Validate the date range
print(target_time_series_df.index.min())
print(target_time_series_df.index.max())

In [ ]:
# Restrict the columns to timestamp, traffic_volume
target_time_series_df = target_time_series_df[["traffic_volume"]]
# Add in item_id
target_time_series_df["item_id"] = "all"
# Validate the structure
target_time_series_df.head()

In [ ]:
# With the data in a great state, save it off as a CSV
target_time_series_filename = "target_time_series.csv"
target_time_series_path = f"{data_dir}/{target_time_series_filename}"
target_time_series_df.to_csv(target_time_series_path, header=False)

### Building The Validation File

This is the last file we need to build before getting started with Forecast itself. This will be the same in structure as our target-time-series file but will only project into 2018 and includes no historical data from the training data.

In [ ]:
validation_time_series_df = target_df.copy()
validation_time_series_df = validation_time_series_df.loc["2018-01-01":]
# Validate the date range
print(validation_time_series_df.index.min())
print(validation_time_series_df.index.max())

In [ ]:
# Restrict the columns to timestamp, traffic_volume
validation_time_series_df = validation_time_series_df[["traffic_volume"]]
# Add in item_id
validation_time_series_df["item_id"] = "all"
# Validate the structure
validation_time_series_df.head()

In [ ]:
# With the data in a great state, save it off as a CSV
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = f"{data_dir}/{validation_time_series_filename}"
validation_time_series_df.to_csv(validation_time_series_path, header=False)

## Getting Started With Forecast

Now that all of the required data to get started exists, our next step is to build the dataset groups and datasets required for our problem. Inside Amazon Forecast a DatasetGroup is an abstraction that contains all the datasets for a particular collection of Forecasts. There is no information sharing between DatasetGroups so if you'd like to try out various alternatives to the schemas we create below, you could create a new DatasetGroup and make your changes inside its corresponding Datasets.

The order of the process below will be as follows:

1. Create a DatasetGroup for our POC.
1. Create a `Target-Time-Series` Dataset.
1. Attach the Dataset to the DatasetGroup.
1. Import the data into the Dataset.

Later you can use the other notebooks to build Predictors based off this information, or to add related time series data as well.

First, we'll re-load our environment configuration and connect to the AWS APIs as we did in [0.Environment_Setup.ipynb](0.Environment_Setup.ipynb):

In [ ]:
%store -r region
assert isinstance(region, str), "`region` must be a region name string e.g. 'us-east-1'"
%store -r bucket_name
assert isinstance(bucket_name, str), "`bucket_name` must be a data bucket name string"
%store -r forecast_role_arn
assert isinstance(forecast_role_arn, str), "`forecast_role_arn` must be an IAM role ARN (string)"

session = boto3.Session(region_name=region)
forecast = session.client("forecast")
forecast_query = session.client("forecastquery")
s3 = session.resource("s3")

Next we define a few core aspects of our Dataset Group and info on our data. For example the timestamp format, the project name, and how frequent our time series data is.

In [ ]:
import secrets
import string

DATASET_FREQUENCY = "H" 
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"

project = "forecast_poc_{}".format(
    # Add a random suffix for uniqueness:
    "".join(secrets.choice(string.ascii_lowercase + string.digits) for i in range(4))
)
%store project
datasetName = project + "_ds"
%store datasetName
datasetGroupName = project + "_dsg"
%store datasetGroupName

Create the Dataset Group, this is the highest level of abstraction when using Forecast. There is no information sharing between Dataset Groups so if you want to try out new schemas, or completely different datasets for a problem this is a great isolation layer to use.

In [ ]:
# Create the DatasetGroup
create_dataset_group_response = forecast.create_dataset_group(
    DatasetGroupName=datasetGroupName,
    Domain="CUSTOM",
)
datasetGroupArn = create_dataset_group_response["DatasetGroupArn"]
%store datasetGroupArn

In [ ]:
forecast.describe_dataset_group(DatasetGroupArn=datasetGroupArn)

Assuming you made no initial schema changes, the cell below should just be fine. If you have made any alterations, update the cell accordingly then execute it.

In [ ]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema = {
   "Attributes": [
      {
         "AttributeName": "timestamp",
         "AttributeType": "timestamp",
      },
      {
         "AttributeName": "target_value",
         "AttributeType": "float",
      },
      {
         "AttributeName": "item_id",
         "AttributeType": "string",
      },
   ],
}

Inside every DatasetGroup you can have 3 types of additional data:

1. Target Time Series
1. Related Time Series
1. Item Metadata

In this guide we are really only focusing on the target-time-series bit. The cells below will create this container for you and then add it to your DatasetGroup.

In [ ]:
response = forecast.create_dataset(
    Domain="CUSTOM",
    DatasetType="TARGET_TIME_SERIES",
    DatasetName=datasetName,
    DataFrequency=DATASET_FREQUENCY, 
    Schema=schema,
)

In [ ]:
target_datasetArn = response["DatasetArn"]
forecast.describe_dataset(DatasetArn=target_datasetArn)

In [ ]:
# Attach the Dataset to the Dataset Group:
forecast.update_dataset_group(DatasetGroupArn=datasetGroupArn, DatasetArns=[target_datasetArn])

To import our data into Amazon Forecast, it must first be loaded to Amazon S3. For now, we're only loading a single file of target timeseries data.

In [ ]:
# Upload Target File
s3.Bucket(bucket_name).Object(target_time_series_filename).upload_file(target_time_series_path)
target_s3uri = f"s3://{bucket_name}/{target_time_series_filename}"

Now the data exists in S3 with a matching, valid schema defined in Amazon Forecast, the last thing to do is to import it so you can then start actually generating models!

In [ ]:
# Finally we can import the dataset
datasetImportJobName = "DSIMPORT_JOB_TARGET_POC"
ds_import_job_response = forecast.create_dataset_import_job(
    DatasetImportJobName=datasetImportJobName,
    DatasetArn=target_datasetArn,
    DataSource={
        "S3Config": {
            "Path": target_s3uri,
            "RoleArn": forecast_role_arn,
        },
    },
    TimestampFormat=TIMESTAMP_FORMAT,
)

In [ ]:
ds_import_job_arn = ds_import_job_response["DatasetImportJobArn"]
print(ds_import_job_arn)

The import process can take a little time, so the cell below will poll the status every 30 seconds until the import process has completed. From there we will be able to view the metrics on the data and see that it is valid and ready for use.

In [ ]:
def is_import_status_finished(desc):
    status = desc["Status"]
    if status == "ACTIVE":
        return True
    elif status == "CREATE_FAILED":
        raise ValueError(f"Data import failed!\n{desc}")

util.progress.polling_spinner(
    fn_poll_result=lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn),
    fn_is_finished=is_import_status_finished,
    fn_stringify_result=lambda d: d["Status"],
    poll_secs=30,
    timeout_secs=60*60,  # Max 1 hour
)
print("Data imported")

Once the import shows a state of `ACTIVE` we are then ready to evaluate the data that exists within the system and call the importing process complete.

## Evaluating the Target Time Series Data

First let us take a look at the information provided in our target time series file:

In [ ]:
# Validate the date range
print(target_time_series_df.index.min())
print(target_time_series_df.index.max())

In [ ]:
# Take a look at high level metrics:
target_time_series_df.info()

There are exactly 10,642 entries in this file with no null values at all. Let us now look at the metrics from the import.

In [ ]:
forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)

At long last we see the same metrics from our import that we saw from our dataframe. From here we can now consider our work on target-time-series done. 

If you are running the POC process this is a great time to now explore sorting out the related data bits as well. If you are not you can move on to just building Predictors.

The final cell below will use the store function of jupyter to save off a few variables for use in other notebooks.

In [ ]:
%store full_historical_df
%store target_time_series_df
%store validation_time_series_df
%store target_datasetArn
%store target_time_series_filename
%store target_df
%store full_df
%store DATASET_FREQUENCY
%store TIMESTAMP_FORMAT
